# How to do Almost Everything (to Administer Db2) via SQL and Jupyter Notebook
This Notebook is designed to share the SQL and python code related to the presentation titled How to do Almost Everything (to Administer Db2) via SQL and Jupyter Notebook

## Disclaimer
Run at your own risk. Understand what each cell is doing before executing it. This is not a perfect document or perfect code. Depending on your system, some parts may or may not work. There are no guarantees or support. Interpretation of a skilled Db2 DBA is required.

## License
This notebook is covered under a GNU General Public License. Details are available at https://github.com/ecrooks/db2_and_jupyter_notebooks/blob/master/LICENSE.txt

## Instructions for running
1. Install Jupyter Notebook using Anaconda (https://www.anaconda.com/distribution/)
    - Anaconda can be installed on your laptop or a vm on your laptop - anywhere you can connect to the databases in question. This works well if you are working with it alone, or have to connect to different vpns to connect to different databases
    - Anaconda can be installed on a central VM or server that can connect to the databases you wish to work with. This works well if you are working with a team of DBAs and only need to work with databases on that one network. Anaconda works just fine on Ubuntu if you are looking for a free option
    - Anaconda can be installed directly on the database server. This is generally my last choice, as I would rather not run an http server on my database server. I also rarely only care about one database server.
1. Copy this notebook to the computer you've installed Jupyter Notebook on. I'll refer to this as your Jupyter Notebook server. 
1. Create a separate file to store enviornment variables. I've called mine ember_variables.py, and I run it in a cell below. This allows you to easily share the notebook without also sharing your ids and passwords and other sensative information. This also makes using git or other source control easy so you can keep notebooks updated across multiple locations. The format for this file is laid out below.

### Format for variables file
The ember_variables.py file has a format like this:
```python
NA1_User='yourid'
NA1_PW='yourpw'

NA1_Host='server1.example.com'
NA1_insts = ('db2inst1', 'db2inst2', 'db2inst3', 'db2inst4')
NA1_ports = {'db2inst1': 50001, 'db2inst2': 50002, 'db2inst3':50003, 'db2inst4':50004}
NA1_dbs = {'db2inst1': ['SAMPLE1'], 'db2inst2': ['SAMPLE2'], 'db2inst3':['SAMPLE3'], 'db2inst4':['SAMPLE4','SAMPLE5']}
```
Feel free to structure things differently and if you have any good ideas in this area, please share them.

## Set up the enviornment
### Install Libraries
Run the following cell if it is the first time using this notebook on a specific jupyter notebook server. If anything is installed, restart the kernel using the 'Kernel' menu at the top of this notebook

In [ ]:
import sys,os,os.path
os.environ['IBM_DB_HOME']='C:\Program Files\IBM\SQLLIB'

# Check to see if the libraries already have been installed
import importlib

# Check for ibm_db_sa.  If it exists, it's safe to assume that the other requirements
# are already installed.
spec = importlib.util.find_spec("ibm_db_sa")
if spec is None:
    print("Installing prerequisites.")
    !pip install ipython-sql
    !pip install "ibm-db==2.0.8a"
    !pip install ibm_db_sa
else:
    print("sql magic, ibm_db and ibm_db_sa already installed.")
spec = importlib.util.find_spec("jupyter_contrib_nbextensions")
if spec is None:
    print("Installing prerequisites.")
    !pip install jupyter_contrib_nbextensions
    !pip install jupyter_nbextensions_configurator
else:
    print("jupyter_contrib_nbextensions is already installed.")
spec = importlib.util.find_spec("sqlparse")
if spec is None:
    print("Installing prerequisites.")
    !pip install sqlparse
else:
    print("sqlparse already installed.")

Restart the Kernel if this is your first time installing any of the above. The next steps will fail unless you do this.

### Import the modules and load the SQL magic
Required each time the kernel for this notebook is started or restarted

In [ ]:
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd
from IPython.display import display, HTML, Markdown
import nbextensions
import sqlparse

%matplotlib inline

import getpass

### Set Basic Variables and Connect to Database
Connect to the database. Change the values in your variables file to match the environment you're connecting to. The format for this file is provided above.

In [ ]:
# Define filename for passwords
filename = 'ember_variables.py'
# source the file
%run $filename

In [ ]:
# This is the database connection Cell
# It will prompt the user for a PW. PWs should not be stored.
user=local_User
host=local_Host
inst='db2inst1'

db=local_dbs[inst][0]
port=local_ports[inst]

password = getpass.getpass('Enter password for '+user)

%sql db2+ibm_db://$user:$password@$host:$port/$db

In [ ]:
#Configure SQL Magic in a few nice ways
%config SqlMagic.style = 'MSWORD_FRIENDLY'
pd.set_option('max_rows', 4096)
pd.set_option('max_columns', 4096)

In [ ]:
# Functions that may be used in other cells
def highlight_equals(s,threshold,column):
    is_max = pd.Series(data=False, index=s.index)
    is_max[column] = s.loc[column] == threshold
    print(type(is_max))
    return ['background-color: yellow' if is_max.any() else '' for v in is_max]

## Investigating Database-Level Authorities and Permissions

### Finding Users with an Authority

In [ ]:
%%sql connectauth << select grantee 
from syscat.dbauth 
where connectauth='Y'

In [ ]:
display(connectauth)

### Finding Authorities for a User the Old-Fashioned Way

In [ ]:
%%sql id_auth << select dbadmauth
	, connectauth
	, dataaccessauth 
from syscat.dbauth 
where grantee='USER1'

In [ ]:
display(id_auth)

### Finding Table Permissions for a User the Old-Fashioned Way

In [ ]:
%%sql obj_perms_by_id << select substr(tabschema,1,8) as tabschema
	, substr(tabname,1,18) as tabname
	, controlauth
	, deleteauth
	, insertauth
	, selectauth
	, updateauth 
from syscat.tabauth 
where grantee='DB2INST1'

In [ ]:
display(obj_perms_by_id)

### Finding Users Who Have Permissions on a Table

In [ ]:
%%sql obj_perms_by_obj << select substr(grantee,1,8) as grantee
	, controlauth
	, deleteauth
	, insertauth
	, selectauth
	, updateauth 
from syscat.tabauth 
where tabschema='DB2INST1' 
	and tabname='SALES'

In [ ]:
display(obj_perms_by_obj)

### Listing Privileges by User the Easy Way - Querying Object Permissions

In [ ]:
%%sql obj_perms << select * 
from sysibmadm.privileges 
where authid='DB2INST1'
    and objecttype!='DB2 PACKAGE'

In [ ]:
display(obj_perms)

### Listing Groups a User is a Member Of

In [ ]:
%%sql groups_for_id << select * from
    table(AUTH_LIST_GROUPS_FOR_AUTHID('DB2INST1'))

In [ ]:
display(groups_for_id)

### Listing Authorities for an ID

In [ ]:
%%sql auths_for_id << select * from
    table(AUTH_LIST_AUTHORITIES_FOR_AUTHID('DB2INST1','U'))

In [ ]:
display(auths_for_id)

### Complicated SQL for Nicely Formatted Output

In [ ]:
%%sql db_auth_ids << with tab_perms as(
select rtrim(grantee) as grantee
    , granteetype
    , count(*) as tab_write_access
from syscat.tabauth ta
where ta.updateauth='Y' 
    or ta.insertauth='Y' 
    or ta.deleteauth='Y'
group by grantee, granteetype
) ,
tab_select as(
select rtrim(grantee) as grantee
    , granteetype
    , count(*) as tab_read_access
from syscat.tabauth ta
where ta.selectauth='Y' 
group by grantee, granteetype
)
select coalesce(da.grantee, tp.grantee) as grantee
    , coalesce(da.granteetype, tp.granteetype) as granteetype
    , securityadmauth
    , dbadmauth
    , dataaccessauth
    , case dataaccessauth when 'Y' then (select count(*) from syscat.tables where type in ('T','V')) else tp.tab_write_access end as tab_write_access
    , case dataaccessauth when 'Y' then (select count(*) from syscat.tables where type in ('T','V')) else ts.tab_read_access end as tab_read_access
from syscat.dbauth da
    left outer join tab_perms tp on da.grantee=tp.grantee and da.granteetype=tp.granteetype
    left outer join tab_select ts on da.grantee=ts.grantee and da.granteetype=ts.granteetype
where securityadmauth='Y' or dbadmauth='Y' or dataaccessauth='Y' or tab_write_access > 0 or tab_read_access > 0
order by granteetype desc, securityadmauth desc, dbadmauth desc, dataaccessauth desc, tab_write_access desc, tab_read_access desc
with ur

In [ ]:
display(db_auth_ids)

## Table Investigation Section

### Unused Tables

In [ ]:
%%sql unused_tables <<     select  t.lastused
        , date(stats_time) as stats_time
        , date(create_time) as create_time
        , t.tabschema,1,10
        , t.tabname,1,25
        , bigint(card) as table_card
        , mt.table_scans
        , mt.rows_read
        , mt.rows_inserted + mt.rows_updated + mt.rows_deleted as rows_altered
        , t.volatile 
        , mt.member
    from    syscat.tables t 
        join table(mon_get_table('','',-2)) as mt on t.tabschema=mt.tabschema and t.tabname = mt.tabname 
    where 
        t.tabschema not like 'SYS%' 
        and t.tabname not like '%EXPLAIN%' 
        and t.tabname not like '%ADVISE%' 
        and t.lastused < current date - 30 days 
        and type = 'T' 
        and stats_time is not null and length(t.tabname) < 15 and t.tabschema != 'DBAMON097'
    order by t.lastused, t.card desc, t.tabschema, t.tabname 
    with ur

In [ ]:
display(unused_tables)

### Busiest Tables

In [ ]:
%%sql mostused_tables <<  select  t.lastused 
        , substr(t.tabschema,1,10) as tabschema
        , substr(t.tabname,1,25) as tabname
        , bigint(card) as table_card 
        , mt.table_scans
        , mt.rows_read
        , case when card >0 then mt.rows_read/card else 0 end as avg_reads_per_row
        , mt.rows_inserted + mt.rows_updated + mt.rows_deleted as rows_altered
        , t.volatile 
        , mt.member
    from    syscat.tables t 
        join table(mon_get_table('','',-2)) as mt on t.tabschema=mt.tabschema and t.tabname = mt.tabname
    where 
        t.tabschema not like 'SYS%' 
        and t.tabname not like '%EXPLAIN%' 
        and t.tabname not like '%ADVISE%' 
    order by 7 desc, 8 desc, t.tabschema, t.tabname, mt.member 
    fetch first 20 rows only 
    with ur

In [ ]:
busy_tab_df=mostused_tables.DataFrame()
busy_tab_df['table_card'] = busy_tab_df.apply(lambda x: "{:,}".format(x['table_card']), axis=1)
busy_tab_df['table_scans'] = busy_tab_df.apply(lambda x: "{:,}".format(x['table_scans']), axis=1)
busy_tab_df['rows_read'] = busy_tab_df.apply(lambda x: "{:,}".format(x['rows_read']), axis=1)
busy_tab_df['avg_reads_per_row'] = busy_tab_df.apply(lambda x: "{:,}".format(x['avg_reads_per_row']), axis=1)
busy_tab_df['rows_altered'] = busy_tab_df.apply(lambda x: "{:,}".format(x['rows_altered']), axis=1)
display(HTML(busy_tab_df.to_html(index=False)))


### Largest Tables by Size 

In [ ]:
%%sql largest_tables_by_size << select  t.lastused, 
        substr(t.tabschema,1,10) as tabschema, 
        substr(t.tabname,1,25) as tabname, 
        bigint(card) as table_card, 
        data_object_p_size/1024 as data_size_mb, 
        index_object_p_size/1024 as index_size_mb, 
        lob_object_p_size/1024 as lob_size_mb, 
        (ati.data_object_p_size + index_object_p_size + long_object_p_size + lob_object_p_size + xml_object_p_size + col_object_p_size)/1024 as size_mb, 
        (select listagg(colname ,chr(10)) within group (order by colno) from syscat.columns c where c.tabschema=t.tabschema and c.tabname=t.tabname and typename in ('DATE','TIMESTAMP')) as date_cols, 
        (select listagg(colname ,chr(10)) within group (order by colno) from syscat.columns c where c.tabschema=t.tabschema and c.tabname=t.tabname and typename like '%LOB') as lob_cols, 
        t.volatile 
from    syscat.tables t 
        join table(mon_get_table('','',-2)) as mt on t.tabschema=mt.tabschema and t.tabname = mt.tabname 
        join sysibmadm.admintabinfo ati on t.tabschema=ati.tabschema and t.tabname=ati.tabname 
where 
        t.tabschema not like 'SYS%' 
        and t.tabname not like '%EXPLAIN%' 
        and t.tabname not like '%ADVISE%' 
order by size_mb desc, t.tabschema, t.tabname 
fetch first 20 rows only 
with ur

In [ ]:
display(largest_tables_by_size)

## MON_GET* 

### Finding Problem SQL in MON_GET_PKG_CACHE_STMT

In [ ]:
%%sql prob_sql << WITH SUM_TAB (SUM_RR, SUM_CPU, SUM_EXEC, SUM_SORT, SUM_NUM_EXEC) AS ( 
        SELECT  nullif(FLOAT(SUM(ROWS_READ)),0), 
                nullif(FLOAT(SUM(TOTAL_CPU_TIME)),0), 
                nullif(FLOAT(SUM(STMT_EXEC_TIME)),0), 
                nullif(FLOAT(SUM(TOTAL_SECTION_SORT_TIME)),0), 
                nullif(FLOAT(SUM(NUM_EXECUTIONS)),0) 
            FROM TABLE(MON_GET_PKG_CACHE_STMT ( 'D', NULL, NULL, -2)) AS T 
            WHERE stmt_text not like '%monreport.dbsummary%'
        ) 
SELECT substr(stmt_text,1,25) as STATEMENT, 
        ROWS_READ, 
        coalesce(DECIMAL(100*(FLOAT(ROWS_READ)/SUM_TAB.SUM_RR),5,2),0) AS PCT_TOT_RR, 
        TOTAL_CPU_TIME, 
        coalesce(DECIMAL(100*(FLOAT(TOTAL_CPU_TIME)/SUM_TAB.SUM_CPU),5,2),0) AS PCT_TOT_CPU, 
        STMT_EXEC_TIME, 
        coalesce(DECIMAL(100*(FLOAT(STMT_EXEC_TIME)/SUM_TAB.SUM_EXEC),5,2),0) AS PCT_TOT_EXEC_TIME, 
        TOTAL_SECTION_SORT_TIME, 
        coalesce(DECIMAL(100*(FLOAT(TOTAL_SECTION_SORT_TIME)/SUM_TAB.SUM_SORT),5,2),0) AS PCT_TOT_SRT, 
        NUM_EXECUTIONS, 
        coalesce(DECIMAL(100*(FLOAT(NUM_EXECUTIONS)/SUM_TAB.SUM_NUM_EXEC),5,2),0) AS PCT_TOT_EXECS, 
        DECIMAL(FLOAT(STMT_EXEC_TIME)/FLOAT(NUM_EXECUTIONS),10,2) AS AVG_EXEC_TIME, 
        INSERT_TIMESTAMP,
        hex(EXECUTABLE_ID) as EXECUTABLE_ID,
        RTRIM(STMT_TEXT) as FULL_STATEMENT 
    FROM TABLE(MON_GET_PKG_CACHE_STMT ( 'D', NULL, NULL, -2)) AS T, SUM_TAB 
    WHERE (DECIMAL(100*(FLOAT(ROWS_READ)/SUM_TAB.SUM_RR),5,2) > 10 
            OR DECIMAL(100*(FLOAT(TOTAL_CPU_TIME)/SUM_TAB.SUM_CPU),5,2) >10 
            OR DECIMAL(100*(FLOAT(STMT_EXEC_TIME)/SUM_TAB.SUM_EXEC),5,2) >10 
            OR DECIMAL(100*(FLOAT(TOTAL_SECTION_SORT_TIME)/SUM_TAB.SUM_SORT),5,2) >10 
            OR DECIMAL(100*(FLOAT(NUM_EXECUTIONS)/SUM_TAB.SUM_NUM_EXEC),5,2) >10 )
        AND stmt_text not like '%monreport.dbsummary%'
    ORDER BY ROWS_READ DESC 
    FETCH FIRST 20 ROWS ONLY 
    WITH UR

In [ ]:
df=prob_sql.DataFrame()
#df = pd.read_csv(r"C:\Users\ecrooks\Documents\GitHub\private_jupyter_notebooks\problem_sql.csv")

#display(df.columns)
df[['pct_tot_rr']]=df[['pct_tot_rr']].astype(float)
df[['pct_tot_cpu']]=df[['pct_tot_cpu']].astype(float)
df[['pct_tot_exec_time']]=df[['pct_tot_exec_time']].astype(float)
df[['pct_tot_srt']]=df[['pct_tot_srt']].astype(float)
df[['pct_tot_execs']]=df[['pct_tot_execs']].astype(float)
df[['avg_exec_time']]=df[['avg_exec_time']].astype(float)

df['rows_read'] = df['rows_read'].map(lambda x: '{:,}'.format(x))
df['total_cpu_time'] = df['total_cpu_time'].map(lambda x: '{:,}'.format(x))
df['stmt_exec_time'] = df['stmt_exec_time'].map(lambda x: '{:,}'.format(x))
df['total_section_sort_time'] = df['total_section_sort_time'].map(lambda x: '{:,}'.format(x))
df['num_executions'] = df['num_executions'].map(lambda x: '{:,}'.format(x))

#pd.options.display.float_format = '{:,.2f}'.format
display(df[['STATEMENT','rows_read','pct_tot_rr','total_cpu_time','pct_tot_cpu','stmt_exec_time','pct_tot_exec_time','total_section_sort_time','pct_tot_srt','num_executions','pct_tot_execs','avg_exec_time']])
#df.plot(x='STATEMENT', y=['pct_tot_rr','pct_tot_cpu','pct_tot_exec_time','pct_tot_srt'], kind='barh')
#plt.show


In [ ]:
pos=len(df)
df_add=df
df_add.loc[pos] = pd.Series('OTHER', index = ['STATEMENT'])
df_add.at[pos, 'pct_tot_rr'] = 100 - df['pct_tot_rr'].sum()
df_add.at[pos, 'pct_tot_cpu'] = 100 - df['pct_tot_cpu'].sum()
df_add.at[pos, 'pct_tot_exec_time'] = 100 - df['pct_tot_exec_time'].sum()
df_add.at[pos, 'pct_tot_srt'] = 100 - df['pct_tot_srt'].sum()
df_add.at[pos, 'pct_tot_execs'] = 100 - df['pct_tot_execs'].sum()
df_add['query_num'] = df_add.index
df_add['query_num']=df_add['query_num'].apply(lambda x: '{0:0>2}'.format(x))
df_add[['query_num']]= 'query' + df[['query_num']]
df_add.at[pos, 'query_num'] = 'other'

#display(df_add)
df_add2=df_add.drop(['rows_read', 'total_cpu_time', 'stmt_exec_time', 'total_section_sort_time', 'num_executions', 'avg_exec_time', 'insert_timestamp', 'executable_id', 'STATEMENT', 'full_statement'], axis=1)
#display(df_add2)
df_add3=df_add2.set_index('query_num').T

df_add3.rename(index={'pct_tot_rr':'Rows Read'},inplace=True)
df_add3.rename(index={'pct_tot_cpu':'CPU Time'},inplace=True)
df_add3.rename(index={'pct_tot_exec_time':'Execution Time'},inplace=True)
df_add3.rename(index={'pct_tot_srt':'Sort Time'},inplace=True)
df_add3.rename(index={'pct_tot_execs':'Number of Executions'},inplace=True)
#display(df_add3)

In [ ]:
ax = df_add3.plot(kind='barh', title ="Percent of Resource Consumption by Top Problem Queries",figsize=(15,10),legend=True, stacked=True, fontsize=12, colormap='Paired')

plt.show

In [ ]:
conn=%sql
#display(conn)
schema='wscomusr'
pd.set_option('display.max_colwidth', -1)
for index, row in df.iterrows():
    # skip the "other" row added to balance out numbers for the metrics
    if row['query_num'] == 'other': 
        continue
    # Display basic information about the query
    display(Markdown("## Query "+str(index)))
    display(Markdown("### Query Characteristics"))
    display(Markdown("Executed "+str(row['num_executions'])+" times since last placed in the package cache at "+str(row['insert_timestamp'])))
    display(Markdown("Consumed "+str(row['pct_tot_rr'])+" percent of all rows read by all queries in the package cache."))
    display(Markdown("Consumed "+str(row['pct_tot_cpu'])+" percent of all cpu time used by all queries in the package cache."))
    display(Markdown("Consumed "+str(row['pct_tot_exec_time'])+" percent of all execution time used by all queries in the package cache."))
    display(Markdown("Consumed "+str(row['pct_tot_srt'])+" percent of all sort time used by all queries in the package cache."))
    display(Markdown("### Query Text"))
    formatted_sql=sqlparse.format(df['full_statement'][index], reindent=True)
    print(formatted_sql.replace("\\n","<br>"))
    # If a database connection is available, gather additional information about this query
    ## Note: explain may fail if the interval between runing the query to find problem sql and this section was too long, and the section has been cleared from the package cache
    if conn:
        #When db connection is available
        display(Markdown("### Query Explain Plan"))
        display(row.dtypes)
        exe_id=row['executable_id']
        ex_schema=user.upper()
        ex_requester=''
        ex_time=''
        src_name=''
        src_schema=schema
        src_version=''
        %sql call explain_from_section(x'{exe_id}', 'M', NULL, 0, :ex_schema, :ex_requester, :ex_time, :src_name, :src_schema, :src_version)
        expln_plan=%sql select * from {user}.last_explained
        print(expln_plan)

## Diagnostic Log and History Section

In [ ]:
%%sql backup_list << select date(timestamp(start_time)) as start_date 
    , time(timestamp(start_time)) as start_time 
    , start_time as start_timestamp 
    , dayname(start_time) as day
    , timestampdiff ( 4, varchar(timestamp(end_time) - timestamp(start_time)) ) as duration 
    , case operationtype 
        when 'D' then 'Delta Offline' 
        when 'E' then 'Delta Online' 
        when 'F' then 'Offline' 
        when 'I' then 'Incremental Offline' 
        when 'N' then 'Online' 
        when 'O' then 'Incremental Online' 
     else operationtype 
     end || ' ' || case 
            when objecttype = 'D' then 'DB' 
            when objecttype = 'P' then 'TS'
            else objecttype 
        end as Type 
    , devicetype 
    , sqlcode 
from sysibmadm.db_history 
where operation='B' 
    and start_time > current timestamp - 14 days
order by start_date, start_time 
with ur 

In [ ]:
display(backup_list)

In [ ]:
%%sql diag_log << SELECT TIMESTAMP
    , substr(APPL_ID,1,15) as APPL_ID_TRUNC
    , MSGSEVERITY as SEV
    , MSGNUM
    , substr(MSG,1,50) as MSG_trunc 
FROM TABLE ( PD_GET_LOG_MSGS( CURRENT_TIMESTAMP - 7 DAYS)) AS T 
ORDER BY TIMESTAMP DESC

In [ ]:
display(diag_log)

## Querying System and Database Configuration Information

### Server Configuration
Note: may not work how you expect it to on docker

In [ ]:
%%sql system_info << SELECT OS_NAME 
    , HOST_NAME 
    , OS_FULL_VERSION 
    , OS_KERNEL_VERSION 
    , OS_ARCH_TYPE 
    , CPU_TOTAL 
    , CPU_ONLINE 
    , CPU_CONFIGURED 
    , CPU_SPEED 
    , CPU_HMT_DEGREE 
    , CPU_CORES_PER_SOCKET 
    , MEMORY_TOTAL 
    , MEMORY_FREE 
    , VIRTUAL_MEM_TOTAL
    , VIRTUAL_MEM_RESERVED 
    , VIRTUAL_MEM_FREE 
    , CPU_LOAD_SHORT 
    , CPU_LOAD_MEDIUM 
    , CPU_LOAD_LONG 
    , CPU_USAGE_TOTAL 
    , CPU_USER 
    , CPU_IDLE 
    , CPU_IOWAIT 
    , CPU_SYSTEM 
    , SWAP_PAGE_SIZE 
    , SWAP_PAGES_IN 
    , SWAP_PAGES_OUT 
FROM TABLE(SYSPROC.ENV_GET_SYSTEM_RESOURCES()) AS T 
with ur

In [ ]:
display(system_info)

### Db2 Version

In [ ]:
%%sql vers_info << SELECT INST_NAME 
    , IS_INST_PARTITIONABLE 
    , NUM_DBPARTITIONS 
    , INST_PTR_SIZE 
    , RELEASE_NUM 
    , SERVICE_LEVEL 
    , BLD_LEVEL 
    , PTF 
    , FIXPACK_NUM 
    , NUM_MEMBERS 
FROM SYSIBMADM.ENV_INST_INFO 
with ur

In [ ]:
display(vers_info)

### Db2 License Information

In [ ]:
%%sql lic_info << SELECT INSTALLED_PROD 
    , INSTALLED_PROD_FULLNAME 
    , LICENSE_INSTALLED 
    , PROD_RELEASE 
    , LICENSE_TYPE 
from SYSIBMADM.ENV_PROD_INFO 
with ur

In [ ]:
lic_info_df=lic_info.DataFrame()
lic_info_df.style.apply(highlight_equals,threshold='Y',column=['license_installed'], axis=1)

### Db2 Registry

In [ ]:
%%sql reg_info << SELECT DBPARTITIONNUM
    , REG_VAR_NAME 
    , REG_VAR_VALUE 
    , IS_AGGREGATE 
    , AGGREGATE_NAME 
    , LEVEL 
from SYSIBMADM.REG_VARIABLES
order by DBPARTITIONNUM, REG_VAR_NAME
with ur

In [ ]:
display(reg_info)

### DBM Configuration

In [ ]:
%%sql dbm_cfg << SELECT NAME 
    , VALUE 
    , VALUE_FLAGS 
    , DEFERRED_VALUE 
    , DEFERRED_VALUE_FLAGS 
from SYSIBMADM.DBMCFG 
with ur

In [ ]:
display(dbm_cfg)

### DB Configuration

In [ ]:
%%sql db_cfg << SELECT NAME 
        , VALUE 
        , VALUE_FLAGS 
        , DEFERRED_VALUE 
        , DEFERRED_VALUE_FLAGS 
    from SYSIBMADM.DBCFG 
    with ur

In [ ]:
display(db_cfg)

### Querying DB Storage Layout

In [ ]:
%%sql db_paths << select *
from sysibmadm.dbpaths

In [ ]:
display(db_paths)